# Kanazawa AI Meetup, 2018

## Automatic Differentiation Tutorial
- Anand Krish

In [1]:
import numpy as np
import json

# Database for precomputed derivatives for elementary functions 
# (Add your own gradients here)
G = {
     "add":lambda a,b: a+b,
     "sub":lambda a,b: a-b,
     "mul":lambda a,b: a*b,
     "square":lambda  a: a**2,
     "log":lambda a:np.log(a),
     "sin":lambda a:np.sin(a),
     }

DG = {
    "add":[(lambda a,b: 1),(lambda a,b: 1)],
    "sub":[(lambda a,b: 1),(lambda a,b: -1)],
    "mul":[(lambda a,b: b),(lambda a,b: a)],
    "square":[(lambda  a: 2*a)],
    "log":[(lambda  a: 1/a)],
    "sin":[(lambda  a: np.cos(a))],
    }

In [2]:
## Reverse Accumulation Code
# Forward Pass
def eval(f,val):
    for t in f:
        op = G[t[1]]    # Get the lambda function for the corresponding function name
        var = list(map(val.get, t[2])) # Get the values of variables from the dict
        val[t[0]] = op(*var)  # Obtain the result of the function for the list of values
    return val["f"]     # Return the final value

# Reverse pass (back propagate the gradients from the output to the input)
def rev_acc(f,val):
    delta["f"] = 1 # Output jitter
    for t in reversed(f): # Scan from the output to the input
        var = list(map(val.get, t[2])) 
        for i in range(len(t[2])): # Perform backprop for each variable in the function
            op = DG[t[1]][i]   # Obtain the precomputer gradient
            delta[t[2][i]] += delta[t[0]]*op(*var) # standard error update rule

In [3]:
# f = (2x1*x2 + x2)^2
val = {"x1":3, "x2":7}
delta = {"x1":0,"x2":0}
inter_val = {"z1":0,"z2":0,"z3":0,"f":0}
f = [("z1","add", ["x1","x1"]),
     ("z2","mul", ["x2","z1"]),
     ("z3", "add", ["x2", "z2"]),
     ("f", "square", ["z3"])]


val = {**val, **inter_val}
delta = {**delta, **inter_val}

## One forward pass and one reverse pass
eval(f,val)
# Ensure whether all the variables are computed
print("Forward Pass for f")
print(json.dumps(val, indent=4))
# Perform Reverse Mode Autodiff
rev_acc(f,val)
print("Reverse Pass for f")
print(json.dumps(delta, indent=4))

Forward Pass for f
{
    "x1": 3,
    "x2": 7,
    "z1": 6,
    "z2": 42,
    "z3": 49,
    "f": 2401
}
Reverse Pass for f
{
    "x1": 1372,
    "x2": 686,
    "z1": 686,
    "z2": 98,
    "z3": 98,
    "f": 1
}


In [4]:
# f = log(x1) +x1x2 - sin(x2)
val = {"x1":2, "x2":5}
delta = {"x1":0,"x2":0}
inter_val = {"z1":0,"z2":0,"z3":0,"z4":0,"f":0}
f = [("z1","log", ["x1"]),
     ("z2","mul", ["x2","x1"]),
     ("z3", "sin", ["x2"]),
     ("z4", "add", ["z1","z2"]),
     ("f", "sub", ["z4","z3"])]

val = {**val, **inter_val}
delta = {**delta, **inter_val}

## One forward pass and one reverse pass
eval(f,val)
# Ensure whether all the variables are computed
print("Forward Pass for f")
print(json.dumps(val, indent=4))
# Perform Reverse Mode Autodiff
rev_acc(f,val)
print("Reverse Pass for f")
print(json.dumps(delta, indent=4))

Forward Pass for f
{
    "x1": 2,
    "x2": 5,
    "z1": 0.6931471805599453,
    "z2": 10,
    "z3": -0.9589242746631385,
    "z4": 10.693147180559945,
    "f": 11.652071455223084
}
Reverse Pass for f
{
    "x1": 5.5,
    "x2": 1.7163378145367738,
    "z1": 1,
    "z2": 1,
    "z3": -1,
    "z4": 1,
    "f": 1
}


In [6]:
# f = [x^2, log(2x)]
val = {"x1":3}
delta = {"x1":0}
inter_val = {"z2":0,"f":0}
f1 = [("f", "square", ["x1"])]
f2 = [("z2","add", ["x1","x1"]),
     ("f", "log", ["z2"])]

val = {**val, **inter_val}
delta = {**delta, **inter_val}

## Two forward passes and two reverse passes
# Reverse accumulation w.r.t. f1
eval(f1,val)
# Ensure whether all the variables are computed
print("Forward Pass for f1")
print(json.dumps(val, indent=4))
# Perform Reverse Mode Autodiff
rev_acc(f1,val)
print("Reverse Pass for f1")
print(json.dumps(delta, indent=4))
print("-----------------------------------------")
# Reverse accumulation w.r.t. f2
eval(f2,val)
# Ensure whether all the variables are computed
print("Forward Pass for f2")
print(json.dumps(val, indent=4))
# Perform Reverse Mode Autodiff
rev_acc(f2,val)
print("Reverse Pass for f2")
print(json.dumps(delta, indent=4))

Forward Pass for f1
{
    "x1": 3,
    "z2": 0,
    "f": 9
}
Reverse Pass for f1
{
    "x1": 6,
    "z2": 0,
    "f": 1
}
-----------------------------------------
Forward Pass for f2
{
    "x1": 3,
    "z2": 6,
    "f": 1.791759469228055
}
Reverse Pass for f2
{
    "x1": 6.333333333333334,
    "z2": 0.16666666666666666,
    "f": 1
}
